# Sector Coupling in PyPSA

This tutorial covers how to model sector coupling in PyPSA, integrating different energy sectors (power, heat, transport) and their interactions. We'll explore how different sectors can be connected and optimized together.

## Sector Coupling Overview

Sector coupling involves connecting different energy sectors:

1. **Power Sector**: Electricity generation and distribution
2. **Heat Sector**: Heating and cooling
3. **Transport Sector**: Electric vehicles and other transport modes

Let's create a network that demonstrates sector coupling:

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create network with time series
network = pypsa.Network()
network.set_snapshots(pd.date_range("2024-01-01", periods=24, freq="H"))

# Add buses for different sectors
network.add("Bus", "electricity", v_nom=20.0)
network.add("Bus", "heat", carrier="heat")
network.add("Bus", "transport", carrier="transport")
network.add("Bus", "hydrogen", carrier="hydrogen")

# Add carriers
network.add("Carrier", "gas", co2_emissions=0.2)
network.add("Carrier", "solar")
network.add("Carrier", "heat")
network.add("Carrier", "transport")
network.add("Carrier", "hydrogen")

## Adding Sector-Specific Components

Let's add components for each sector:

In [ ]:
# Power sector components
network.add(
    "Generator",
    "solar",
    bus="electricity",
    p_nom=100,
    p_max_pu=pd.Series(
        [0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8,
         0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
        index=network.snapshots
    ),
    carrier="solar"
)

# Heat sector components
network.add(
    "Generator",
    "gas_boiler",
    bus="heat",
    p_nom=50,
    efficiency=0.9,
    carrier="gas"
)

# Transport sector components
network.add(
    "Load",
    "ev_charging",
    bus="transport",
    p_set=20,
    p_max_pu=1.5,  # Maximum charging power
    p_min_pu=0.0   # Can be zero when not charging
)

## Adding Sector Coupling Technologies

Now let's add technologies that connect different sectors:

In [ ]:
# Power-to-Heat: Heat pump
network.add(
    "Link",
    "heat_pump",
    bus0="electricity",
    bus1="heat",
    p_nom=30,
    efficiency=3.0,  # Coefficient of Performance
    capital_cost=800
)

# Power-to-Transport: EV charging
network.add(
    "Link",
    "ev_charger",
    bus0="electricity",
    bus1="transport",
    p_nom=20,
    efficiency=0.95,
    capital_cost=500
)

# Power-to-Hydrogen: Electrolyzer
network.add(
    "Link",
    "electrolyzer",
    bus0="electricity",
    bus1="hydrogen",
    p_nom=50,
    efficiency=0.7,
    capital_cost=1000
)

# Hydrogen-to-Transport: Fuel cell vehicles
network.add(
    "Link",
    "h2_vehicle",
    bus0="hydrogen",
    bus1="transport",
    p_nom=30,
    efficiency=0.6,
    capital_cost=1200
)

## Adding Storage and Flexibility Options

Let's add storage and flexibility options for each sector:

In [ ]:
# Battery storage for power sector
network.add(
    "Store",
    "battery",
    bus="electricity",
    e_nom=100,
    e_cyclic=True,
    p_nom=20,
    capital_cost_e=100,
    capital_cost_p=1000
)

# Heat storage
network.add(
    "Store",
    "heat_storage",
    bus="heat",
    e_nom=200,
    e_cyclic=True,
    p_nom=40,
    standing_loss=0.01,
    capital_cost_e=50,
    capital_cost_p=500
)

# Hydrogen storage
network.add(
    "Store",
    "h2_storage",
    bus="hydrogen",
    e_nom=500,
    e_cyclic=True,
    p_nom=50,
    standing_loss=0.001,
    capital_cost_e=200,
    capital_cost_p=2000
)

## Adding Global Constraints

Let's add some global constraints for the coupled system:

In [ ]:
# CO2 emission limit
network.add(
    "GlobalConstraint",
    "co2_limit",
    sense="<=",
    constant=1000  # Maximum CO2 emissions in tCO2
)

# Renewable share target
network.add(
    "GlobalConstraint",
    "renewable_share",
    sense=">=",
    constant=0.5  # 50% renewable share
)

## Optimizing the Coupled System

Let's solve the optimization problem and analyze the results:

In [ ]:
# Solve the optimization problem
network.lopf()

# Plot power flows between sectors
plt.figure(figsize=(12, 6))
network.links_t.p0.plot()
plt.title('Power Flows Between Sectors')
plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.legend(title='Link')
plt.grid(True)
plt.show()

# Plot storage operation
plt.figure(figsize=(12, 6))
network.stores_t.e.plot()
plt.title('Storage Energy Levels')
plt.xlabel('Time')
plt.ylabel('Energy (MWh)')
plt.legend(title='Storage')
plt.grid(True)
plt.show()

## Key Sector Coupling Concepts

1. **Sector Integration**
   - Multiple energy carriers
   - Cross-sector technologies
   - Shared infrastructure

2. **Coupling Technologies**
   - Power-to-Heat (heat pumps)
   - Power-to-Transport (EV charging)
   - Power-to-Hydrogen (electrolysis)
   - Hydrogen-to-Transport (fuel cells)

3. **Storage Options**
   - Electrical storage (batteries)
   - Thermal storage
   - Chemical storage (hydrogen)

4. **Flexibility**
   - Demand-side management
   - Storage operation
   - Cross-sector optimization

## Benefits of Sector Coupling

1. **Increased Flexibility**
   - More options for balancing supply and demand
   - Better integration of variable renewables
   - Enhanced system resilience

2. **Cost Optimization**
   - Shared infrastructure
   - Better utilization of assets
   - Reduced system costs

3. **Environmental Benefits**
   - Higher renewable share
   - Reduced CO2 emissions
   - Sustainable energy system

## Next Steps

This completes the PyPSA tutorial series. You now have a comprehensive understanding of how to use PyPSA for power system analysis, from basic network creation to complex sector coupling scenarios.